In [27]:
from zenrows import ZenRowsClient
from bs4 import BeautifulSoup

client = ZenRowsClient("e60f3e2234cfaec6a5530d270606277b656cd797")
url = "https://jiji.co.ke/kileleshwa/houses-apartments-for-rent/1bdrm-apartment-in-kileleshwa-for-rent-pgXaP0IDanUfU7Rgn9H5xbNa.html"
# url = "https://jiji.co.ke/shanzu/temporary-and-vacation-rentals/2-bedroom-beachfront-airbnb-apartment-in-mombasa-1zXUskgyApokEmxT4dQvX1q8.html"
#params = {"js_render":"true","premium_proxy":"true"}
params = {"js_render":"true"}
response = client.get(url, params=params, timeout=120)

soup = BeautifulSoup(response.text, 'lxml')


In [28]:
[i.text for i in soup.select("div.b-advert-attribute__key")]

['Property Address', 'Property Size', 'Condition', 'Furnishing', 'Toilets']

In [29]:
[i.text for i in soup.select("div.b-advert-attribute__value")]

['Kileleshwa', '82sqm', 'Fairly Used', 'Unfurnished', '1']

In [30]:
description = soup.select_one("span.qa-description-text").text
currency = soup.select_one('meta[itemprop="priceCurrency"]')['content'] if soup.select_one('meta[itemprop="priceCurrency"]') else None
price = soup.select_one('div[itemprop="price"]')['content'] if soup.select_one('div[itemprop="price"]') else None
images = [i.select_one('img')['src'] for i in soup.select("source[type='image/webp']")]
agent = soup.select_one("div.b-seller-block__name").text

print(agent, currency, price, images, description)

EdenWorld Properties KES 60000 ['https://pictures-kenya.jijistatic.com/72810434_MTYwMC0xNjAwLTJkNDg1MDNiYjc.webp', 'https://pictures-kenya.jijistatic.com/72810434_MzAwLTMwMC0yZDQ4NTAzYmI3.webp', 'https://pictures-kenya.jijistatic.com/72810381_MzAwLTMwMC05NzE5OGU5YTE0.webp', 'https://pictures-kenya.jijistatic.com/72810390_MzAwLTMwMC01NzI4N2NiNGRj.webp', 'https://pictures-kenya.jijistatic.com/72810402_MzAwLTMwMC1mNzU2OGNiNDVm.webp', 'https://pictures-kenya.jijistatic.com/72810420_MzAwLTMwMC00N2Y2ZjFkZjIw.webp'] Executive 1 Bedroom Apartments To Let 
Kileleshwa
 
Unit immediately available on 6th Floor 
Amenities
• Fitted Gym 
• Swimming pool
• High Speed Lifts 
• Spacious Balconies 
• Video Intercom 
• 24/H Security 
• Backup Generator 
• Borehole
• Electric Fence


In [31]:
import re

keys = [i.text for i in soup.select("div.b-advert-attribute__key")]
values = [i.text for i in soup.select("div.b-advert-attribute__value")]

key_value_map = dict(zip(keys, values))

propertyType = key_value_map.get("Property Type", None) or key_value_map.get("Type", None)
constructionStatus = key_value_map.get("Status of Construction", None)

property_size_value = key_value_map.get("Property Size") or key_value_map.get("Square Metres") or key_value_map.get("Land Area")
if property_size_value:
    size_match = re.match(r"([\d.]+)([a-zA-Z]+)", property_size_value)
    size = size_match.group(1) if size_match else None
    sizeUnit = size_match.group(2).strip() if size_match else None
else:
    size = None
    sizeUnit = None

beds = key_value_map.get("Number of Bedrooms", None)
baths = key_value_map.get("Number of Bathrooms", None)
toilets = key_value_map.get("Toilets", None)
parking = key_value_map.get("Parking Spaces", None)
address = key_value_map.get("Address", None) or key_value_map.get("Property Address", None)
amenities = (
    key_value_map.get("Facilities", "").split(", ") 
    if key_value_map.get("Facilities") 
    else [
        amenity.text.strip() 
        for amenity in soup.select("div.b-advert-attributes__tag") 
        if amenity and amenity.text
    ]
)
description = soup.select_one("span.qa-description-text").text if soup.select_one("span.qa-description-text") else None
currency = soup.select_one('meta[itemprop="priceCurrency"]')['content'] if soup.select_one('meta[itemprop="priceCurrency"]') else None
price = soup.select_one('div[itemprop="price"]')['content'] if soup.select_one('div[itemprop="price"]') else None
images = [i.select_one('img')['src'] for i in soup.select("source[type='image/webp']")] if soup.select("source[type='image/webp']") else []
agent = soup.select_one("div.b-seller-block__name").text if soup.select_one("div.b-seller-block__name") else None

print(propertyType, constructionStatus, size, sizeUnit, beds, baths, toilets, parking, address, amenities)


None None 82 sqm None None 1 None Kileleshwa []


In [32]:
import re

if (len(soup.select("div.b-advert-icon-attribute__image")) == 3):

    propertyType = soup.select("div.b-advert-icon-attribute__image")[0].find_next_sibling().text

    bed_number_match = re.match(r'\d+', soup.select("div.b-advert-icon-attribute__image")[1].find_next_sibling().text)
    beds = bed_number_match.group() if bed_number_match else None

    bath_number_match = re.match(r'\d+', soup.select("div.b-advert-icon-attribute__image")[2].find_next_sibling().text)
    baths = bath_number_match.group() if bath_number_match else None

if (len(soup.select("div.b-advert-icon-attribute__image")) == 2):
    bed_number_match = re.match(r'\d+', soup.select("div.b-advert-icon-attribute__image")[0].find_next_sibling().text)
    beds = bed_number_match.group() if bed_number_match else None

    bath_number_match = re.match(r'\d+', soup.select("div.b-advert-icon-attribute__image")[1].find_next_sibling().text)
    baths = bath_number_match.group() if bath_number_match else None

In [33]:
price = float(soup.select_one('div[itemprop="price"]')['content']) if soup.select_one('div[itemprop="price"]') else None
currency = soup.select_one('meta[itemprop="priceCurrency"]')['content'] if soup.select_one('meta[itemprop="priceCurrency"]') else None
pricingCriteria = soup.select_one('span.b-alt-advert-price__period').text if soup.select_one('span.b-alt-advert-price__period') else None

print(price, currency, pricingCriteria)

60000.0 KES per month


In [ ]:
import os, io, logging, boto3
import time
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
from pymongo import MongoClient, UpdateOne
from zenrows import ZenRowsClient

# Load environment variables
load_dotenv(override=True)
CONNECTION_STRING_MONGODB = os.getenv("CONNECTION_STRING")
aws_region_name = os.getenv("aws_region_name")
bucket_name = os.getenv("bucket_name")
zenRowsApiKey = os.getenv("ZENROWS_API_KEY")
threads = int(os.getenv("threads"))
databaseName = 'jiji_com_gh'

# Configure logging

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S')

log = logging.getLogger("jiji-co-tz-url-extractor")
log_stringio = io.StringIO()
handler = logging.StreamHandler(log_stringio)
handler.setFormatter(formatter)
log.addHandler(handler)


# ZenRows client
client = ZenRowsClient(zenRowsApiKey)
params = {"js_render":"true"}

def sendData(data, databaseName, collectionName):
    try:
        client = MongoClient(CONNECTION_STRING_MONGODB)
        db = client[databaseName]
        collection = db[collectionName]

        operations = [
            UpdateOne({'propertyId': record['propertyId']}, {'$set': record}, upsert=True)
            for record in data
        ]
        if operations:
            collection.bulk_write(operations)
        logging.info(f"Sent {len(data)} records to MongoDB.")
    except Exception as e:
        logging.error(f"Error sending data to MongoDB: {e}")

def process_adverts(data):
    return [
        {
            'url': f"https://jiji.com.gh{i.get('url')}",
            'propertyId': i.get('id'),
            'price': i.get('price_obj').get('value'),
            'user_phone': i.get('user_phone'),
            'title': i.get('title'),
            'city': i.get('region_parent_name'),
            'neighborhood': i.get('region_name'),
        }
        for i in data
    ]

def fetch_with_retries(url, params, retries=3, timeout=120):
    for attempt in range(retries):
        try:
            response = client.get(url, params=params, timeout=timeout)
            return response.json()
        except Exception as e:
            log.warning(f"Error fetching URL: {url}, attempt {attempt + 1}/{retries}: {e}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt)
    log.error(f"Failed to fetch URL: {url} after {retries} attempts.")
    return None

def fetch_data(slug):
    all_data = []
    url = f"https://jiji.com.gh/api_web/v1/listing?slug={slug}"
    print('Processing url:', url)
    # Initial fetch
    response = fetch_with_retries(url, params=params)
    if response:
        data = response.get('adverts_list', {}).get('adverts', [])
        all_data = process_adverts(data)
        # Handle pagination with retries
        while response.get('next_url', None):
            next_url = response.get('next_url')
            print('Processing url:', next_url)
            response = fetch_with_retries(next_url, params=params)
            if response:
                data = response.get('adverts_list', {}).get('adverts', [])
                all_data += process_adverts(data)
            else:
                log.error(f"Failed to fetch paginated data for slug {slug}. Skipping remaining pages.")
                break  # Exit pagination loop if retries fail

    return all_data

allSlugs = ['new-builds', 'event-centers-and-venues', 'land-and-plots-for-rent', 'commercial-properties', 'commercial-property-for-rent', 'temporary-and-vacation-rentals', 'houses-apartments-for-sale', 'houses-apartments-for-rent', 'land-and-plots-for-sale']

log.info('Gathering property links !')
with ThreadPoolExecutor(max_workers=threads) as executor:
    results = executor.map(fetch_data, allSlugs)
    for result in results:
        sendData(result, databaseName, 'propertyURLs')

    log.info("URL extraction completed successfully.")
    s3 = boto3.client("s3", region_name=aws_region_name)
    s3.put_object(Body=log_stringio.getvalue(), Bucket=bucket_name, Key="logs/jiji_com_gh/url-extractor-logs.txt")
    log.info("Logs transferred to s3 completed")


In [ ]:
import requests, json

apikey = "e60f3e2234cfaec6a5530d270606277b656cd797"
url = "https://www.buyrentkenya.com/listings/1-bedroom-apartment-for-sale-westlands-area-3766711"
response = requests.get("https://api.zenrows.com/v1/", timeout=120, params={
	"apikey": apikey,
	"url": url,
})
soup = BeautifulSoup(response.text, 'lxml')

In [79]:
housingType = soup.select_one("ul li.whitespace-nowrap:nth-of-type(2)").text.strip() if soup.select_one("ul li.whitespace-nowrap:nth-of-type(2)") else None
city = soup.select_one('nav[data-cy="breadcrumbs"] ul li:nth-of-type(4)').text.strip() if soup.select_one('nav[data-cy="breadcrumbs"] ul li:nth-of-type(4)') else None
suburb = soup.select_one('nav[data-cy="breadcrumbs"] ul li:nth-of-type(5)').text.strip() if soup.select_one('nav[data-cy="breadcrumbs"] ul li:nth-of-type(5)') else None
title = soup.select_one("h1[data-cy='listing-heading']").text.strip() if soup.select_one("h1[data-cy='listing-heading']") else None
location = soup.select_one('p[data-cy="listing-address"]').text.strip() if soup.select_one('p[data-cy="listing-address"]') else None
beds = int(soup.select_one('span[aria-label*="bedrooms"]').text.strip().replace('\n', '')) if soup.select_one('span[aria-label*="bedrooms"]') else None
baths = int(soup.select_one('span[aria-label*="bathrooms"]').text.strip().replace('\n', '')) if soup.select_one('span[aria-label*="bathrooms"]') else None
size = float(soup.select_one('span[aria-label*="area"]').text.strip().split()[0]) if soup.select_one('span[aria-label*="area"]') else None
parking = bool(soup.select_one('ul li div div:contains("Parking")')) if soup.select_one('ul li div div:contains("Parking")') else False
amenities = [item.text.replace('\n', '').replace('|', '').strip() for item in soup.select('div[data-cy="listing-amenities-component"] li')]
imgUrls = [img['src'] for img in soup.select('#gallery_slider img')]

data = json.loads(soup.select_one('div[data-bi="product-top"]').get('wire:snapshot')) if soup.select_one('div[data-bi="product-top"]') else json.loads(soup.select_one('div[data-bi="product-premium"]').get('wire:snapshot')) if soup.select_one('div[data-bi="product-premium"]') else json.loads(soup.select_one('div[data-bi="product-basic"]').get('wire:snapshot'))

In [ ]:
data = json.loads(soup.select_one('div[data-bi="product-top"]').get('wire:snapshot')) if soup.select_one('div[data-bi="product-top"]') else json.loads(soup.select_one('div[data-bi="product-premium"]').get('wire:snapshot')) if soup.select_one('div[data-bi="product-premium"]') else json.loads(soup.select_one('div[data-bi="product-basic"]').get('wire:snapshot'))

description = data['data']['listingResult'][0]['_source'][0]['description']
try:
    sourcePricePerSf = data['data']['listingResult'][0]['_source'][0]['data_properties'][0]['price_per_sq_metre']
    unit = data['data']['listingResult'][0]['_source'][0]['data_properties'][0]['area_unit']

    if "square-meters" in unit:
        sourcePricePerSf = round(sourcePricePerSf / 10.7639, 2)
    elif 'acres' in unit:
        sourcePricePerSf = round(sourcePricePerSf / 43560, 2)
    else:
        sourcePricePerSf = round(sourcePricePerSf, 2)
except KeyError:
    sourcePricePerSf, unit = None, None

agent = data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['name']
agentNumber = data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['chat_link'].replace("https://wa.me/", '').replace('?text=', '') if data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['chat_link'] else data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['mobile_number']

None None


In [91]:
agent = data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['name']
agentNumber = data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['chat_link'].replace("https://wa.me/", '').replace('?text=', '') if data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['chat_link'] else data['data']['listingResult'][0]['_source'][0]['agents'][0][0][0]['mobile_number']

print(agent, agentNumber)

Round the Clock Homes +254723898445
